### System Installation
Installing the coref tool from mandarjoshi90 along with tensorflow.




In [ ]:
!nvidia-smi --query-gpu=gpu_name,driver_version,memory.total --format=csv

In [ ]:
! python --version

In [ ]:
! git clone https://github.com/mandarjoshi90/coref.git

In [ ]:
%cd coref
! sed 's/MarkupSafe==1.0/MarkupSafe==1.1.1/; s/scikit-learn==0.19.1/scikit-learn==0.21/; s/scipy==1.0.0/scipy==1.6.2/' < requirements.txt > tmp
! mv tmp requirements.txt

! sed 's/.D.GLIBCXX.USE.CXX11.ABI.0//' < setup_all.sh  > tmp
! mv tmp setup_all.sh 
! chmod u+x setup_all.sh 

In [ ]:
%tensorflow_version 2.x
! pip uninstall -y tensorflow
! pip install -r requirements.txt --log install-log.txt -q
! ./setup_all.sh

### Specifying Input

Input and Model

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
genre = "nw"
# The Ontonotes data for training the model contains text from several sources
# of very different styles. You need to specify the most suitable one out of:
# "bc": broadcast conversation
# "bn": broadcast news
# "mz": magazine
# "nw": newswire
# "pt": Bible text
# "tc": telephone conversation
# "wb": web data

model_name = "spanbert_base"
# The fine-tuned model to use. Options are:
# bert_base
# spanbert_base
# bert_large
# spanbert_large

In [ ]:
import os
os.environ['data_dir'] = "./data"
os.environ['CHOSEN_MODEL'] = model_name

Downloading the selected model.

In [ ]:
! ./download_pretrained.sh $CHOSEN_MODEL

## Splitting into chunks and merging.
Choose the directory for the data. 
The input should be in the CoNLL format.
Preferably without discontinous spans

In [ ]:
import os

# Change for each set.
output = "./data/dev.english.v4_gold_conll"
input = "../CRAFT-conll/dev"

# Number of sentences to split on.
N = 200

def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

# Clears the current file from data.
open(output, 'w').close()

for file in [f for f in os.listdir(input) if f.endswith('.conll')]:

  with open(input + "/" + file) as f_in:
    
    first_line = f_in.readline()

    paragraphs = [line.split('\n') for line in f_in.read().split('\n\n') if line]

    chs = chunks(paragraphs, N)

    with open(output, "a") as f_out:
      
      for idx, ch in enumerate(chs):
        
        f_out.write(first_line[0:-2] + str(idx) + "\n")
        
        for paragraph in ch:
          for line in paragraph:
            parts = line.split()
            parts[1] = idx
            f_out.write(' '.join([str(p) for p in parts[0:2]]) + ' ' + '\t'.join([str(p) for p in parts[2:]]) + "\n")
          f_out.write('\n')
        f_out.write('\n')

In [ ]:
! ./setup_training.sh $data_dir $data_dir

## Choose vocab file and data directory. 

Generates JSONline files from conll files. Change in experiments file for file names etc.

In [ ]:
! python minimize.py ./data/cased_L-24_H-1024_A-16/vocab.txt ./data/ ./data/ true

Trains a new model. Choose starting model. Models without train prefix is finetuned on OntoNotes

In [ ]:
! GPU=0 python train.py train_spanbert_base

### Evaluating
Choose the model to evaluate. The evaluation script provided offical CoNLL-2012 score.


In [ ]:
! gpu=0 python evaluate.py $CHOSEN_MODEL